In [0]:
!pip install xlrd==1.2.0
!pip install pyxlsb

Requirement already satisfied: xlrd==1.2.0 in /databricks/python3/lib/python3.8/site-packages (1.2.0)
 WARNING: You are using pip version 21.0.1; however, version 22.0.4 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command. 
Requirement already satisfied: pyxlsb in /databricks/python3/lib/python3.8/site-packages (1.0.9)
 WARNING: You are using pip version 21.0.1; however, version 22.0.4 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.

In [0]:
import pandas as pd
import numpy as np
import xlrd
import datetime
from datetime import date, timedelta
import urllib.parse
import urllib.request
import json
import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [0]:
#Analytical dataset
model_data = spark.read.table('model_data_vf_19_01')  #enter table name
model_data = model_data.toPandas()
model_data = model_data[model_data['PO_Create_Date']!='1888-01-01']
model_data = model_data[model_data['Delivered_Quantity']!=0]

#Daily weather dataset import
weather_raw_data=spark.read.table('weather_data_weekly') 
weather_raw_data_v1 = weather_raw_data.toPandas()

#Extreme weather events dataset
#1) extreme weather events based on region, city
#2) extreme weather events based on ports in costal cities 
exw_vendors_within_US = spark.read.table('Extreme_weather_data_for_vendors_within_US')  #enter table name
exw_vendors_within_US = exw_vendors_within_US.toPandas()

exw_vendors_outside_US = spark.read.table('Extreme_weather_data_for_vendors_outside_US')   #enter table name
exw_vendors_outside_US = exw_vendors_outside_US.toPandas()

all_event_types = spark.read.table('extreme_weather_event_types')   #enter table name
all_event_types = all_event_types.toPandas()

#Traffic dataset
#1) traffic data based on region, city
#2) traffic data based on ports in costal cities
traffic_data= spark.read.table("Traffic_dataset_raw")  #enter table name
traffic_data_final=traffic_data.toPandas()

#Port congestion dataset
port_data = spark.read.table('Port_congestion_data')#enter table name
port_data = port_data.toPandas()     

In [0]:
# updating values on model data 
for vs_id, vs_reg in zip(model_data['updated_VS_location'],model_data['updated_VS_region']):
  if (((vs_id=='madison') | (vs_id=='payette')) & (vs_reg=='id')):
    model_data.loc[(model_data['updated_VS_location']==vs_id) & (model_data['updated_VS_region']==vs_reg),'updated_VS_country']='us'
  else:
    pass

model_data.loc[model_data['updated_VS_region']=='de','updated_VS_country']='de'
model_data.loc[model_data['updated_VS_region']=='br','updated_VS_country']='br'
model_data.loc[model_data['updated_VS_region']=='vn','updated_VS_country']='vt'
model_data.loc[model_data['updated_VS_country']=='vt','updated_VS_region']='vt'

In [0]:
Model_wth_daily_weather_events="model_weather_data_v1"
Model_with_weather_dataset="Model_with_weather_data"
Model_with_weather_traffic_dataset="Model_with_weather_traffic_data"
Model_with_weather_traffic_port_dataset="Model_weather_traffic_port_congestion_data_v1"

In [0]:
# creating forecasting timeline of 2,4,6 weeks from po create date for building the leadtime prediction models 

model_data_v1 = model_data[['Plant_ID', 'Plant_Name', 'updated_VS_ID', 'updated_VS_location', 'updated_VS_region', 
                            'updated_VS_country', 'Purchase_Order', 'PO_Create_Date', 'Delivery_date', 'lead_time']]

model_data_v1['PO_create_date_2W']=pd.to_datetime(model_data_v1['PO_Create_Date'])+timedelta(14)
model_data_v1['PO_create_date_2W']=pd.to_datetime(model_data_v1['PO_create_date_2W']).dt.date
model_data_v1['PO_create_date_2W']=pd.to_datetime(model_data_v1['PO_create_date_2W'])
model_data_v1['PO_create_date_4W']=pd.to_datetime(model_data_v1['PO_Create_Date'])+timedelta(28)
model_data_v1['PO_create_date_4W']=pd.to_datetime(model_data_v1['PO_create_date_4W']).dt.date
model_data_v1['PO_create_date_4W']=pd.to_datetime(model_data_v1['PO_create_date_4W'])
model_data_v1['PO_create_date_6W']=pd.to_datetime(model_data_v1['PO_Create_Date'])+timedelta(42)
model_data_v1['PO_create_date_6W']=pd.to_datetime(model_data_v1['PO_create_date_6W']).dt.date
model_data_v1['PO_create_date_6W']=pd.to_datetime(model_data_v1['PO_create_date_6W'])
model_data_v1['name'] = model_data_v1['updated_VS_location']+','+model_data_v1['updated_VS_region']
model_data_v1['Weeks']=pd.to_datetime(model_data_v1['PO_Create_Date']).dt.week
print(model_data_v1.shape)
model_data_v1.head()

(69351, 15)
Out[6]:

,Plant_ID,Plant_Name,updated_VS_ID,updated_VS_location,updated_VS_region,updated_VS_country,Purchase_Order,PO_Create_Date,Delivery_date,lead_time,PO_create_date_2W,PO_create_date_4W,PO_create_date_6W,name,Weeks
0,5259,us pl little chute,100935737,wabasha,mn,us,4565760881,2021-04-05,2021-05-06,31,2021-04-19,2021-05-03,2021-05-17,"wabasha,mn",14
1,5259,us pl little chute,100935737,wabasha,mn,us,4565760889,2021-03-19,2021-04-18,30,2021-04-02,2021-04-16,2021-04-30,"wabasha,mn",11
2,5259,us pl little chute,100935737,wabasha,mn,us,4565762950,2021-03-26,2021-04-27,32,2021-04-09,2021-04-23,2021-05-07,"wabasha,mn",12
3,5259,us pl little chute,100935737,wabasha,mn,us,4565768904,2021-03-26,2021-04-30,35,2021-04-09,2021-04-23,2021-05-07,"wabasha,mn",12
4,5259,us pl little chute,100935737,wabasha,mn,us,4565788542,2021-03-11,2021-04-08,28,2021-03-25,2021-04-08,2021-04-22,"wabasha,mn",10


In [0]:
#creating weather_type attributes for 2, 4, and 6 weeks for building the leadtime prediction models
weather_event_types=['rainfall_mm','snowdepth','tempmax','tempmin','windspeed','fog_count','snow_count','rain_count']
for event_type in weather_event_types:
  templist1=[]
  templist2=[]
  templist3=[]  
  for names,po_date_week,po_date_2w,po_date_4w,po_date_6w in zip(model_data_v1['name'],model_data_v1['PO_Create_Date'].dt.week,model_data_v1['PO_create_date_2W'].dt.week,model_data_v1['PO_create_date_4W'].dt.week,model_data_v1['PO_create_date_6W'].dt.week):
    if event_type!='tempmin':
      if po_date_week<po_date_2w:
        templist2w=weather_raw_data_v1[(weather_raw_data_v1['name']==names) & (weather_raw_data_v1['Weeks']>=po_date_week) & (weather_raw_data_v1['Weeks']<=po_date_2w)][event_type].max()
      else:
        templist2w=weather_raw_data_v1[(weather_raw_data_v1['name']==names) & ((weather_raw_data_v1['Weeks']>=po_date_week) | (weather_raw_data_v1['Weeks']<=po_date_2w))][event_type].max()
      if po_date_week<po_date_4w:
        templist4w=weather_raw_data_v1[(weather_raw_data_v1['name']==names) & (weather_raw_data_v1['Weeks']>=po_date_week) & (weather_raw_data_v1['Weeks']<=po_date_4w)][event_type].max()
      else:
        templist4w=weather_raw_data_v1[(weather_raw_data_v1['name']==names) & ((weather_raw_data_v1['Weeks']>=po_date_week) | (weather_raw_data_v1['Weeks']<=po_date_4w))][event_type].max()
      if po_date_week<po_date_6w:
        templist6w=weather_raw_data_v1[(weather_raw_data_v1['name']==names) & (weather_raw_data_v1['Weeks']>=po_date_week) & (weather_raw_data_v1['Weeks']<=po_date_6w)][event_type].max()
      else:
        templist6w=weather_raw_data_v1[(weather_raw_data_v1['name']==names) & ((weather_raw_data_v1['Weeks']>=po_date_week) | (weather_raw_data_v1['Weeks']<=po_date_6w))][event_type].max()
    else:
      if po_date_week<po_date_2w:
        templist2w=weather_raw_data_v1[(weather_raw_data_v1['name']==names) & (weather_raw_data_v1['Weeks']>=po_date_week) & (weather_raw_data_v1['Weeks']<=po_date_2w)][event_type].min()
      else:
        templist2w=weather_raw_data_v1[(weather_raw_data_v1['name']==names) & ((weather_raw_data_v1['Weeks']>=po_date_week) | (weather_raw_data_v1['Weeks']<=po_date_2w))][event_type].min()
      if po_date_week<po_date_4w:
        templist4w=weather_raw_data_v1[(weather_raw_data_v1['name']==names) & (weather_raw_data_v1['Weeks']>=po_date_week) & (weather_raw_data_v1['Weeks']<=po_date_4w)][event_type].min()
      else:
        templist4w=weather_raw_data_v1[(weather_raw_data_v1['name']==names) & ((weather_raw_data_v1['Weeks']>=po_date_week) | (weather_raw_data_v1['Weeks']<=po_date_4w))][event_type].min()
      if po_date_week<po_date_6w:
        templist6w=weather_raw_data_v1[(weather_raw_data_v1['name']==names) & (weather_raw_data_v1['Weeks']>=po_date_week) & (weather_raw_data_v1['Weeks']<=po_date_6w)][event_type].min()
      else:
        templist6w=weather_raw_data_v1[(weather_raw_data_v1['name']==names) & ((weather_raw_data_v1['Weeks']>=po_date_week) | (weather_raw_data_v1['Weeks']<=po_date_6w))][event_type].min()
    templist1.append(templist2w)
    templist2.append(templist4w)
    templist3.append(templist6w)
  model_data_v1[event_type+'_2W']=templist1
  model_data_v1[event_type+'_4W']=templist2
  model_data_v1[event_type+'_6W']=templist3
  
print(model_data_v1.shape)
model_data_v1.head()

(69351, 39)
Out[7]:

,Plant_ID,Plant_Name,updated_VS_ID,updated_VS_location,updated_VS_region,updated_VS_country,Purchase_Order,PO_Create_Date,Delivery_date,lead_time,PO_create_date_2W,PO_create_date_4W,PO_create_date_6W,name,Weeks,rainfall_mm_2W,rainfall_mm_4W,rainfall_mm_6W,snowdepth_2W,snowdepth_4W,snowdepth_6W,tempmax_2W,tempmax_4W,tempmax_6W,tempmin_2W,tempmin_4W,tempmin_6W,windspeed_2W,windspeed_4W,windspeed_6W,fog_count_2W,fog_count_4W,fog_count_6W,snow_count_2W,snow_count_4W,snow_count_6W,rain_count_2W,rain_count_4W,rain_count_6W
0,5259,us pl little chute,100935737,wabasha,mn,us,4565760881,2021-04-05,2021-05-06,31,2021-04-19,2021-05-03,2021-05-17,"wabasha,mn",14,0.489857,0.907143,3.229429,1.455000,1.455000,1.455000,59.971429,63.785714,69.721429,29.414286,29.414286,29.414286,12.935714,13.585714,13.585714,0.0,0.0,0.0,1.0,1.0,1.0,1.5,2.5,4.5
1,5259,us pl little chute,100935737,wabasha,mn,us,4565760889,2021-03-19,2021-04-18,30,2021-04-02,2021-04-16,2021-04-30,"wabasha,mn",11,1.397000,1.397000,1.397000,1.232857,1.232857,1.455000,53.564286,59.971429,63.785714,30.414286,30.414286,29.414286,13.035714,13.035714,13.035714,0.0,0.0,0.0,3.5,3.5,3.5,2.0,2.0,2.5
2,5259,us pl little chute,100935737,wabasha,mn,us,4565762950,2021-03-26,2021-04-27,32,2021-04-09,2021-04-23,2021-05-07,"wabasha,mn",12,1.397000,1.397000,1.397000,0.249286,1.455000,1.455000,59.971429,59.971429,63.785714,32.842857,29.414286,29.414286,13.035714,13.035714,13.585714,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,2.5
3,5259,us pl little chute,100935737,wabasha,mn,us,4565768904,2021-03-26,2021-04-30,35,2021-04-09,2021-04-23,2021-05-07,"wabasha,mn",12,1.397000,1.397000,1.397000,0.249286,1.455000,1.455000,59.971429,59.971429,63.785714,32.842857,29.414286,29.414286,13.035714,13.035714,13.585714,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,2.5
4,5259,us pl little chute,100935737,wabasha,mn,us,4565788542,2021-03-11,2021-04-08,28,2021-03-25,2021-04-08,2021-04-22,"wabasha,mn",10,1.397000,1.397000,1.397000,2.200526,2.200526,2.200526,45.692857,59.971429,59.971429,26.810526,26.810526,26.810526,13.700000,13.700000,13.700000,0.0,0.0,0.0,3.5,3.5,3.5,2.0,2.0,2.0


In [0]:
#Saving the table with model data and daily weather events
model_data_v1.fillna(0,inplace=True)
col_names = ['_'.join(col.split(' ')) for col in model_data_v1.columns.tolist()]
model_data_v1.columns = col_names
model_with_daily_weather_events = spark.createDataFrame(model_data_v1)
model_with_daily_weather_events.write.saveAsTable(Model_wth_daily_weather_events,mode = 'overwrite')

Combining model with extreme weather events

In [0]:
#creating the month column for both PO_Create_Date & PO_create_date_6W
model_data_v1['PO_month']=pd.to_datetime(model_data_v1['PO_Create_Date']).dt.month
model_data_v1['PO_month_6W']=pd.to_datetime(model_data_v1['PO_create_date_6W']).dt.month

#creating attributes in the model data with null values
for event_type in all_event_types['event_types']:
  model_data_v1[['_'.join(event_type.split(' '))+'_count']]=np.nan
  model_data_v1[['_'.join(event_type.split(' '))+'_duration']]=np.nan
  
#filtering the model data for vendors in the countries US, CA, MX
model_data_first_cut=model_data_v1[(model_data_v1['updated_VS_country']=='us') | (model_data_v1['updated_VS_country']=='ca') | (model_data_v1['updated_VS_country']=='mx')].copy()

#model data with extreme weather events for first cut is generated between po create date and 6 weeks for building the leadtime prediction models
for names,PO_month,PO_month_6W,indexs in zip(model_data_first_cut['name'],model_data_first_cut['PO_month'],model_data_first_cut['PO_month_6W'],model_data_first_cut.index):
  for event_type in all_event_types['event_types']:
    tempdf=pd.DataFrame()
    column1='_'.join(event_type.split(' '))+'_count'
    column2='_'.join(event_type.split(' '))+'_duration'
    if PO_month<PO_month_6W:
      val_df=exw_vendors_within_US[(exw_vendors_within_US['name']==names) & (exw_vendors_within_US['Months']>=PO_month) & (exw_vendors_within_US['Months']<=PO_month_6W)][column1]
    else:
      val_df=exw_vendors_within_US[(exw_vendors_within_US['name']==names) & ((exw_vendors_within_US['Months']>=PO_month) | (exw_vendors_within_US['Months']<=PO_month_6W))][column1]
    val_df=val_df[val_df.isnull()==False]
    val_count=val_df.sum()
    model_data_first_cut[column1][indexs] = val_count
    if PO_month<PO_month_6W:
      tempdf['event_duration']=exw_vendors_within_US[(exw_vendors_within_US['name']==names) & (exw_vendors_within_US['Months']>=PO_month) & (exw_vendors_within_US['Months']<=PO_month_6W)][column2]
    else:
      tempdf['event_duration']=exw_vendors_within_US[(exw_vendors_within_US['name']==names) & ((exw_vendors_within_US['Months']>=PO_month) | (exw_vendors_within_US['Months']<=PO_month_6W))][column2]
    tempdf=tempdf[tempdf['event_duration'].isnull()==False]
    event_duration=tempdf['event_duration'].mean()
    overall_event_duration=event_duration*val_count
    model_data_first_cut[column2][indexs] = overall_event_duration
print(model_data_first_cut.shape)
model_data_first_cut.head()

(68730, 119)
Out[9]:

,Plant_ID,Plant_Name,updated_VS_ID,updated_VS_location,updated_VS_region,updated_VS_country,Purchase_Order,PO_Create_Date,Delivery_date,lead_time,PO_create_date_2W,PO_create_date_4W,PO_create_date_6W,name,Weeks,rainfall_mm_2W,rainfall_mm_4W,rainfall_mm_6W,snowdepth_2W,snowdepth_4W,snowdepth_6W,tempmax_2W,tempmax_4W,tempmax_6W,tempmin_2W,tempmin_4W,tempmin_6W,windspeed_2W,windspeed_4W,windspeed_6W,fog_count_2W,fog_count_4W,fog_count_6W,snow_count_2W,snow_count_4W,snow_count_6W,rain_count_2W,rain_count_4W,rain_count_6W,PO_month,PO_month_6W,extreme_cold/wind_chill_count,extreme_cold/wind_chill_duration,thunderstorm_wind_count,thunderstorm_wind_duration,winter_weather_count,winter_weather_duration,tropical_depression_count,tropical_depression_duration,hurricane_count,hurricane_duration,funnel_cloud_count,funnel_cloud_duration,freezing_fog_count,freezing_fog_duration,cold/wind_chill_count,cold/wind_chill_duration,debris_flow_count,debris_flow_duration,strong_wind_count,strong_wind_duration,high_wind_count,high_wind_duration,lakeshore_flood_count,lakeshore_flood_duration,dense_smoke_count,dense_smoke_duration,frost/freeze_count,frost/freeze_duration,dust_storm_count,dust_storm_duration,tornado_count,tornado_duration,lake-effect_snow_count,lake-effect_snow_duration,storm_surge/tide_count,storm_surge/tide_duration,drought_count,drought_duration,blizzard_count,blizzard_duration,tropical_storm_count,tropical_storm_duration,coastal_flood_count,coastal_flood_duration,wildfire_count,wildfire_duration,high_surf_count,high_surf_duration,dense_fog_count,dense_fog_duration,lightning_count,lightning_duration,excessive_heat_count,excessive_heat_duration,seiche_count,seiche_duration,avalanche_count,avalanche_duration,heavy_rain_count,heavy_rain_duration,ice_storm_count,ice_storm_duration,heavy_snow_count,heavy_snow_duration,winter_storm_count,winter_storm_duration,heat_count,heat_duration,sleet_count,sleet_duration,flash_flood_count,flash_flood_duration,hail_count,hail_duration,rip_current_count,rip_current_duration,flood_count,flood_duration
0,5259,us pl little chute,100935737,wabasha,mn,us,4565760881,2021-04-05,2021-05-06,31,2021-04-19,2021-05-03,2021-05-17,"wabasha,mn",14,0.489857,0.907143,3.229429,1.455000,1.455000,1.455000,59.971429,63.785714,69.721429,29.414286,29.414286,29.414286,12.935714,13.585714,13.585714,0.0,0.0,0.0,1.0,1.0,1.0,1.5,2.5,4.5,4,5,0.0,NaN,0.5,0.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.5,12.0,0.0,NaN,0.0,NaN,0.0,NaN,1.0,0.0,0.0,NaN,0.5,168.0
1,5259,us pl little chute,100935737,wabasha,mn,us,4565760889,2021-03-19,2021-04-18,30,2021-04-02,2021-04-16,2021-04-30,"wabasha,mn",11,1.397000,1.397000,1.397000,1.232857,1.232857,1.455000,53.564286,59.971429,63.785714,30.414286,30.414286,29.414286,13.035714,13.035714,13.035714,0.0,0.0,0.0,3.5,3.5,3.5,2.0,2.0,2.5,3,4,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.5,12.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.5,168.0
2,5259,us pl little chute,100935737,wabasha,mn,us,4565762950,2021-03-26,2021-04-27,32,2021-04-09,2021-04-23,2021-05-07,"wabasha,mn",12,1.397000,1.397000,1.397000,0.249286,1.455000,1.455000,59.971429,59.971429,63.785714,32.842857,29.414286,29.414286,13.035714,13.035714,13.585714,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,2.5,3,5,0.0,NaN,0.5,0.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.5,12.0,0.0,NaN,0.0,NaN,0.0,NaN,1.0,0.0,0.0,NaN,0.5,168.0
3,5259,us pl little chute,100935737,wab

In [0]:
#filtering the model data for vendors not in the countries US, CA, MX
model_data_second_cut=model_data_v1[(model_data_v1['updated_VS_country']!='us') & (model_data_v1['updated_VS_country']!='ca') & (model_data_v1['updated_VS_country']!='mx')].copy()

#creating attribute coast
coast_side=[]
for country in model_data_second_cut['updated_VS_country']:
  if country=='au':
    coast_side.append('West')
  elif country=='tw':
    coast_side.append('West')
  elif country=='es':
    coast_side.append('East')
  elif country=='vt':
    coast_side.append('West')
  elif country=='de':
    coast_side.append('East')
  elif country=='br':
    coast_side.append('East')
model_data_second_cut['Coast']=coast_side

#model data with extreme weather events for second cut is generated between po create date and 6 weeks for building the leadtime prediction models
for Coast,PO_month,PO_month_6W,indexs in zip(model_data_second_cut['Coast'],model_data_second_cut['PO_month'],model_data_second_cut['PO_month_6W'],model_data_second_cut.index):
  for event_type in all_event_types['event_types']:
    tempdf=pd.DataFrame()
    column1='_'.join(event_type.split(' '))+'_count'
    column2='_'.join(event_type.split(' '))+'_duration'
    if PO_month<PO_month_6W:
      val_df=exw_vendors_outside_US[(exw_vendors_outside_US['Coast']==Coast) & (exw_vendors_outside_US['Months']>=PO_month) & (exw_vendors_outside_US['Months']<=PO_month_6W)][column1]
    else:
      val_df=exw_vendors_outside_US[(exw_vendors_outside_US['Coast']==Coast) & ((exw_vendors_outside_US['Months']>=PO_month) | (exw_vendors_outside_US['Months']<=PO_month_6W))][column1]
    val_df=val_df[val_df.isnull()==False]
    val_count=val_df.sum()
    model_data_second_cut[column1][indexs] = val_count
    if PO_month<PO_month_6W:
      tempdf['event_duration']=exw_vendors_outside_US[(exw_vendors_outside_US['Coast']==Coast) & (exw_vendors_outside_US['Months']>=PO_month) & (exw_vendors_outside_US['Months']<=PO_month_6W)][column2]
    else:
      tempdf['event_duration']=exw_vendors_outside_US[(exw_vendors_outside_US['Coast']==Coast) & ((exw_vendors_outside_US['Months']>=PO_month) | (exw_vendors_outside_US['Months']<=PO_month_6W))][column2]
    tempdf=tempdf[tempdf['event_duration'].isnull()==False]
    event_duration=tempdf['event_duration'].mean()
    overall_event_duration=event_duration*val_count
    model_data_second_cut[column2][indexs] = overall_event_duration
model_data_second_cut.drop(columns=['Coast'],inplace=True)

print(model_data_second_cut.shape)
model_data_second_cut.head()

(621, 133)
Out[27]:

,Plant_ID,Plant_Name,updated_VS_ID,updated_VS_location,updated_VS_region,updated_VS_country,Purchase_Order,PO_Create_Date,Delivery_date,lead_time,PO_create_date_2W,PO_create_date_4W,PO_create_date_6W,name,Weeks,rainfall_mm_2W,rainfall_mm_4W,rainfall_mm_6W,snowdepth_2W,snowdepth_4W,snowdepth_6W,tempmax_2W,tempmax_4W,tempmax_6W,tempmin_2W,tempmin_4W,tempmin_6W,windspeed_2W,windspeed_4W,windspeed_6W,fog_count_2W,fog_count_4W,fog_count_6W,snow_count_2W,snow_count_4W,snow_count_6W,rain_count_2W,rain_count_4W,rain_count_6W,PO_month,PO_month_6W,extreme_cold/wind_chill_count,extreme_cold/wind_chill_duration,thunderstorm_wind_count,thunderstorm_wind_duration,winter_weather_count,winter_weather_duration,tropical_depression_count,tropical_depression_duration,hurricane_count,hurricane_duration,funnel_cloud_count,funnel_cloud_duration,freezing_fog_count,freezing_fog_duration,cold/wind_chill_count,cold/wind_chill_duration,debris_flow_count,debris_flow_duration,strong_wind_count,strong_wind_duration,high_wind_count,high_wind_duration,lakeshore_flood_count,lakeshore_flood_duration,dense_smoke_count,dense_smoke_duration,frost/freeze_count,frost/freeze_duration,dust_storm_count,dust_storm_duration,tornado_count,tornado_duration,lake-effect_snow_count,lake-effect_snow_duration,storm_surge/tide_count,storm_surge/tide_duration,drought_count,drought_duration,blizzard_count,blizzard_duration,tropical_storm_count,tropical_storm_duration,coastal_flood_count,coastal_flood_duration,wildfire_count,wildfire_duration,high_surf_count,high_surf_duration,dense_fog_count,dense_fog_duration,lightning_count,lightning_duration,excessive_heat_count,excessive_heat_duration,seiche_count,seiche_duration,avalanche_count,avalanche_duration,heavy_rain_count,heavy_rain_duration,ice_storm_count,ice_storm_duration,heavy_snow_count,heavy_snow_duration,winter_storm_count,winter_storm_duration,heat_count,heat_duration,sleet_count,sleet_duration,flash_flood_count,flash_flood_duration,hail_count,hail_duration,rip_current_count,rip_current_duration,flood_count,flood_duration,marine_tropical_storm_count,marine_tropical_storm_duration,marine_thunderstorm_wind_count,marine_thunderstorm_wind_duration,marine_high_wind_count,marine_high_wind_duration,marine_strong_wind_count,marine_strong_wind_duration,marine_dense_fog_count,marine_dense_fog_duration,waterspout_count,waterspout_duration,marine_hail_count,marine_hail_duration
1821,5259,us pl little chute,100937131,bomaderry,nan,au,4564747931,2020-12-17,2021-02-02,47,2020-12-31,2021-01-14,2021-01-28,"bomaderry,nan",51,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12,1,0.0,NaN,0.0,NaN,0.5,12.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.5,12.0,0.0,NaN,0.0,NaN,8.5,107.1,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.5,0.0,0.0,NaN,0.0,NaN,9.0,600.0,4.0,48.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.5,12.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN
1822,5259,us pl little chute,100937131,bomaderry,nan,au,4564760964,2020-12-11,2021-01-08,28,2020-12-25,2021-01-08,2021-01-22,"bomaderry,nan",50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12,1,0.0,NaN,0.0,NaN,0.5,12.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.5,12.0,0.0,NaN,0.0,NaN,8.5,107.1,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.5,0.0,0.0,NaN,0.0,NaN,9.0,600.0,4.0,48.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.5,12.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN
1823,5259,us pl little chute,100937131,bomaderry,nan,au,4564950663,2020-12-11,2021-01-11,31,2020-12-25,2021-01-08,2021-01-22,"bomaderry,nan",50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12,1,0.0,NaN,0.0,NaN,0.5,12.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.5,12.0,0.0,NaN,0.0,NaN,8.5,107.1,0.0,NaN,0.0

In [0]:
model_data_second_cut.drop(columns=['marine_hail_count','marine_hail_duration','marine_strong_wind_count','marine_strong_wind_duration'],inplace=True)
model_data_second_cut.shape

Out[29]: (621, 129)

In [0]:
model_data_first_cut['marine_dense_fog_count']=0.0
model_data_first_cut['marine_dense_fog_duration']=0.0
model_data_first_cut['marine_high_wind_count']=0.0
model_data_first_cut['marine_high_wind_duration']=0.0
model_data_first_cut['marine_thunderstorm_wind_count']=0.0
model_data_first_cut['marine_thunderstorm_wind_duration']=0.0
model_data_first_cut['marine_tropical_storm_count']=0.0
model_data_first_cut['marine_tropical_storm_duration']=0.0
model_data_first_cut['waterspout_count']=0.0
model_data_first_cut['waterspout_duration']=0.0
model_data_first_cut.shape

Out[30]: (68730, 129)

In [0]:
#model data for building the leadtime prediction models
model_data_v2=pd.concat([model_data_first_cut,model_data_second_cut])

#imputing null values with 0
model_data=model_data_v2.fillna(0)
print(model_data.shape)
model_data.head()

(69351, 129)
Out[31]:

,Plant_ID,Plant_Name,updated_VS_ID,updated_VS_location,updated_VS_region,updated_VS_country,Purchase_Order,PO_Create_Date,Delivery_date,lead_time,PO_create_date_2W,PO_create_date_4W,PO_create_date_6W,name,Weeks,rainfall_mm_2W,rainfall_mm_4W,rainfall_mm_6W,snowdepth_2W,snowdepth_4W,snowdepth_6W,tempmax_2W,tempmax_4W,tempmax_6W,tempmin_2W,tempmin_4W,tempmin_6W,windspeed_2W,windspeed_4W,windspeed_6W,fog_count_2W,fog_count_4W,fog_count_6W,snow_count_2W,snow_count_4W,snow_count_6W,rain_count_2W,rain_count_4W,rain_count_6W,PO_month,PO_month_6W,extreme_cold/wind_chill_count,extreme_cold/wind_chill_duration,thunderstorm_wind_count,thunderstorm_wind_duration,winter_weather_count,winter_weather_duration,tropical_depression_count,tropical_depression_duration,hurricane_count,hurricane_duration,funnel_cloud_count,funnel_cloud_duration,freezing_fog_count,freezing_fog_duration,cold/wind_chill_count,cold/wind_chill_duration,debris_flow_count,debris_flow_duration,strong_wind_count,strong_wind_duration,high_wind_count,high_wind_duration,lakeshore_flood_count,lakeshore_flood_duration,dense_smoke_count,dense_smoke_duration,frost/freeze_count,frost/freeze_duration,dust_storm_count,dust_storm_duration,tornado_count,tornado_duration,lake-effect_snow_count,lake-effect_snow_duration,storm_surge/tide_count,storm_surge/tide_duration,drought_count,drought_duration,blizzard_count,blizzard_duration,tropical_storm_count,tropical_storm_duration,coastal_flood_count,coastal_flood_duration,wildfire_count,wildfire_duration,high_surf_count,high_surf_duration,dense_fog_count,dense_fog_duration,lightning_count,lightning_duration,excessive_heat_count,excessive_heat_duration,seiche_count,seiche_duration,avalanche_count,avalanche_duration,heavy_rain_count,heavy_rain_duration,ice_storm_count,ice_storm_duration,heavy_snow_count,heavy_snow_duration,winter_storm_count,winter_storm_duration,heat_count,heat_duration,sleet_count,sleet_duration,flash_flood_count,flash_flood_duration,hail_count,hail_duration,rip_current_count,rip_current_duration,flood_count,flood_duration,marine_dense_fog_count,marine_dense_fog_duration,marine_high_wind_count,marine_high_wind_duration,marine_thunderstorm_wind_count,marine_thunderstorm_wind_duration,marine_tropical_storm_count,marine_tropical_storm_duration,waterspout_count,waterspout_duration
0,5259,us pl little chute,100935737,wabasha,mn,us,4565760881,2021-04-05,2021-05-06,31,2021-04-19,2021-05-03,2021-05-17,"wabasha,mn",14,0.489857,0.907143,3.229429,1.455000,1.455000,1.455000,59.971429,63.785714,69.721429,29.414286,29.414286,29.414286,12.935714,13.585714,13.585714,0.0,0.0,0.0,1.0,1.0,1.0,1.5,2.5,4.5,4,5,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,12.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.5,168.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5259,us pl little chute,100935737,wabasha,mn,us,4565760889,2021-03-19,2021-04-18,30,2021-04-02,2021-04-16,2021-04-30,"wabasha,mn",11,1.397000,1.397000,1.397000,1.232857,1.232857,1.455000,53.564286,59.971429,63.785714,30.414286,30.414286,29.414286,13.035714,13.035714,13.035714,0.0,0.0,0.0,3.5,3.5,3.5,2.0,2.0,2.5,3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,168.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5259,us pl little chute,100935737,wabasha,mn,us,4565762950,2021-03-26,2021-04-27,32,2021-04-09,2021-04-23,2021-05-07,"wabasha,mn",12,1.397000,1.397000,1.397000,0.249286,1.455000,1.455000,59.971429,59.971429,63.785714,32.842857,29.414286,29.414286,13.035714,13.035714,13.585714,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,2.5,3,5,0.0,0.0,0.5,0.0

In [0]:
#Saving the model data final with extreme weather events
col_names = ['_'.join(col.split(' ')) for col in model_data.columns.tolist()]
model_data.columns = col_names
model_data_save = spark.createDataFrame(model_data)
model_data_save.write.saveAsTable(Model_with_weather_dataset,mode = 'overwrite')

In [0]:
#creating the traffic attributes in model data with null values
for event_type in traffic_data_final['DELAY_TYPE'].unique().tolist():
  model_data[[event_type+'_count_2w']]=np.nan
  model_data[[event_type+'_count_4w']]=np.nan
  model_data[[event_type+'_count_6w']]=np.nan
model_data.head()

#model data with traffic data is generated for 2,4 and 6 weeks for building the leadtime prediction models
for plt_id,vs_id,PO_date,PO_date_2w,PO_date_4w,PO_date_6w,indexs in zip(model_data['Plant_ID'],model_data['updated_VS_ID'],model_data['PO_Create_Date'],model_data['PO_create_date_2W'],model_data['PO_create_date_4W'],model_data['PO_create_date_6W'],model_data.index):
  for event_type in traffic_data_final['DELAY_TYPE'].unique().tolist():
    model_data[event_type+'_count_2w'][indexs]=traffic_data_final[(traffic_data_final['Plant_ID']==plt_id) & (traffic_data_final['updated_VS_ID']==vs_id) & (traffic_data_final['INCIDENT_START_DATE']>=PO_date) & (traffic_data_final['INCIDENT_START_DATE']<=PO_date_2w) & (traffic_data_final['DELAY_TYPE']==event_type)].shape[0]+traffic_data_final[(traffic_data_final['Plant_ID']==plt_id) & (traffic_data_final['updated_VS_ID']==vs_id) & (traffic_data_final['INCIDENT_START_DATE']<=PO_date) & (traffic_data_final['INCIDENT_END_DATE']>=PO_date) & (traffic_data_final['DELAY_TYPE']==event_type)].shape[0]
    
    model_data[event_type+'_count_4w'][indexs]=traffic_data_final[(traffic_data_final['Plant_ID']==plt_id) & (traffic_data_final['updated_VS_ID']==vs_id) & (traffic_data_final['INCIDENT_START_DATE']>=PO_date) & (traffic_data_final['INCIDENT_START_DATE']<=PO_date_4w) & (traffic_data_final['DELAY_TYPE']==event_type)].shape[0]+traffic_data_final[(traffic_data_final['Plant_ID']==plt_id) & (traffic_data_final['updated_VS_ID']==vs_id) & (traffic_data_final['INCIDENT_START_DATE']<=PO_date) & (traffic_data_final['INCIDENT_END_DATE']>=PO_date) & (traffic_data_final['DELAY_TYPE']==event_type)].shape[0]
    
    model_data[event_type+'_count_6w'][indexs]=traffic_data_final[(traffic_data_final['Plant_ID']==plt_id) & (traffic_data_final['updated_VS_ID']==vs_id) & (traffic_data_final['INCIDENT_START_DATE']>=PO_date) & (traffic_data_final['INCIDENT_START_DATE']<=PO_date_6w) & (traffic_data_final['DELAY_TYPE']==event_type)].shape[0]+traffic_data_final[(traffic_data_final['Plant_ID']==plt_id) & (traffic_data_final['updated_VS_ID']==vs_id) & (traffic_data_final['INCIDENT_START_DATE']<=PO_date) & (traffic_data_final['INCIDENT_END_DATE']>=PO_date) & (traffic_data_final['DELAY_TYPE']==event_type)].shape[0]

print(model_data.shape)
model_data.head()

(69351, 138)
Out[33]:

,Plant_ID,Plant_Name,updated_VS_ID,updated_VS_location,updated_VS_region,updated_VS_country,Purchase_Order,PO_Create_Date,Delivery_date,lead_time,PO_create_date_2W,PO_create_date_4W,PO_create_date_6W,name,Weeks,rainfall_mm_2W,rainfall_mm_4W,rainfall_mm_6W,snowdepth_2W,snowdepth_4W,snowdepth_6W,tempmax_2W,tempmax_4W,tempmax_6W,tempmin_2W,tempmin_4W,tempmin_6W,windspeed_2W,windspeed_4W,windspeed_6W,fog_count_2W,fog_count_4W,fog_count_6W,snow_count_2W,snow_count_4W,snow_count_6W,rain_count_2W,rain_count_4W,rain_count_6W,PO_month,PO_month_6W,extreme_cold/wind_chill_count,extreme_cold/wind_chill_duration,thunderstorm_wind_count,thunderstorm_wind_duration,winter_weather_count,winter_weather_duration,tropical_depression_count,tropical_depression_duration,hurricane_count,hurricane_duration,funnel_cloud_count,funnel_cloud_duration,freezing_fog_count,freezing_fog_duration,cold/wind_chill_count,cold/wind_chill_duration,debris_flow_count,debris_flow_duration,strong_wind_count,strong_wind_duration,high_wind_count,high_wind_duration,lakeshore_flood_count,lakeshore_flood_duration,dense_smoke_count,dense_smoke_duration,frost/freeze_count,frost/freeze_duration,dust_storm_count,dust_storm_duration,tornado_count,tornado_duration,lake-effect_snow_count,lake-effect_snow_duration,storm_surge/tide_count,storm_surge/tide_duration,drought_count,drought_duration,blizzard_count,blizzard_duration,tropical_storm_count,tropical_storm_duration,coastal_flood_count,coastal_flood_duration,wildfire_count,wildfire_duration,high_surf_count,high_surf_duration,dense_fog_count,dense_fog_duration,lightning_count,lightning_duration,excessive_heat_count,excessive_heat_duration,seiche_count,seiche_duration,avalanche_count,avalanche_duration,heavy_rain_count,heavy_rain_duration,ice_storm_count,ice_storm_duration,heavy_snow_count,heavy_snow_duration,winter_storm_count,winter_storm_duration,heat_count,heat_duration,sleet_count,sleet_duration,flash_flood_count,flash_flood_duration,hail_count,hail_duration,rip_current_count,rip_current_duration,flood_count,flood_duration,marine_dense_fog_count,marine_dense_fog_duration,marine_high_wind_count,marine_high_wind_duration,marine_thunderstorm_wind_count,marine_thunderstorm_wind_duration,marine_tropical_storm_count,marine_tropical_storm_duration,waterspout_count,waterspout_duration,closed_road_or_infinte_delay_count_2w,closed_road_or_infinte_delay_count_4w,closed_road_or_infinte_delay_count_6w,moderate_count_2w,moderate_count_4w,moderate_count_6w,major_count_2w,major_count_4w,major_count_6w
0,5259,us pl little chute,100935737,wabasha,mn,us,4565760881,2021-04-05,2021-05-06,31,2021-04-19,2021-05-03,2021-05-17,"wabasha,mn",14,0.489857,0.907143,3.229429,1.455000,1.455000,1.455000,59.971429,63.785714,69.721429,29.414286,29.414286,29.414286,12.935714,13.585714,13.585714,0.0,0.0,0.0,1.0,1.0,1.0,1.5,2.5,4.5,4,5,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,12.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.5,168.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5259,us pl little chute,100935737,wabasha,mn,us,4565760889,2021-03-19,2021-04-18,30,2021-04-02,2021-04-16,2021-04-30,"wabasha,mn",11,1.397000,1.397000,1.397000,1.232857,1.232857,1.455000,53.564286,59.971429,63.785714,30.414286,30.414286,29.414286,13.035714,13.035714,13.035714,0.0,0.0,0.0,3.5,3.5,3.5,2.0,2.0,2.5,3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,168.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5259,us pl little chute,100935737,wabasha,mn,us

In [0]:
#saving the model with traffic data
model_data.fillna(0,inplace=True)
col_names = ['_'.join(col.split(' ')) for col in model_data.columns.tolist()]
model_data.columns = col_names
model_data_w_weather_traffic = spark.createDataFrame(model_data)
model_data_w_weather_traffic.write.saveAsTable(Model_with_weather_traffic_dataset,mode = 'overwrite')

In [0]:
model_data_final=model_data.copy()
#creating port congestion attributes in model data with null values
a=['_2w','_4w','_6w']
for week_length in a:
  model_data_final['Port_wait_time_mean'+week_length]=np.nan
  model_data_final['Port_wait_time_median'+week_length]=np.nan
  model_data_final['Port_wait_time_max'+week_length]=np.nan
print(model_data_final.shape)
model_data_final.head()

(69351, 147)
Out[35]:

,Plant_ID,Plant_Name,updated_VS_ID,updated_VS_location,updated_VS_region,updated_VS_country,Purchase_Order,PO_Create_Date,Delivery_date,lead_time,PO_create_date_2W,PO_create_date_4W,PO_create_date_6W,name,Weeks,rainfall_mm_2W,rainfall_mm_4W,rainfall_mm_6W,snowdepth_2W,snowdepth_4W,snowdepth_6W,tempmax_2W,tempmax_4W,tempmax_6W,tempmin_2W,tempmin_4W,tempmin_6W,windspeed_2W,windspeed_4W,windspeed_6W,fog_count_2W,fog_count_4W,fog_count_6W,snow_count_2W,snow_count_4W,snow_count_6W,rain_count_2W,rain_count_4W,rain_count_6W,PO_month,PO_month_6W,extreme_cold/wind_chill_count,extreme_cold/wind_chill_duration,thunderstorm_wind_count,thunderstorm_wind_duration,winter_weather_count,winter_weather_duration,tropical_depression_count,tropical_depression_duration,hurricane_count,hurricane_duration,funnel_cloud_count,funnel_cloud_duration,freezing_fog_count,freezing_fog_duration,cold/wind_chill_count,cold/wind_chill_duration,debris_flow_count,debris_flow_duration,strong_wind_count,strong_wind_duration,high_wind_count,high_wind_duration,lakeshore_flood_count,lakeshore_flood_duration,dense_smoke_count,dense_smoke_duration,frost/freeze_count,frost/freeze_duration,dust_storm_count,dust_storm_duration,tornado_count,tornado_duration,lake-effect_snow_count,lake-effect_snow_duration,storm_surge/tide_count,storm_surge/tide_duration,drought_count,drought_duration,blizzard_count,blizzard_duration,tropical_storm_count,tropical_storm_duration,coastal_flood_count,coastal_flood_duration,wildfire_count,wildfire_duration,high_surf_count,high_surf_duration,dense_fog_count,dense_fog_duration,lightning_count,lightning_duration,excessive_heat_count,excessive_heat_duration,seiche_count,seiche_duration,avalanche_count,avalanche_duration,heavy_rain_count,heavy_rain_duration,ice_storm_count,ice_storm_duration,heavy_snow_count,heavy_snow_duration,winter_storm_count,winter_storm_duration,heat_count,heat_duration,sleet_count,sleet_duration,flash_flood_count,flash_flood_duration,hail_count,hail_duration,rip_current_count,rip_current_duration,flood_count,flood_duration,marine_dense_fog_count,marine_dense_fog_duration,marine_high_wind_count,marine_high_wind_duration,marine_thunderstorm_wind_count,marine_thunderstorm_wind_duration,marine_tropical_storm_count,marine_tropical_storm_duration,waterspout_count,waterspout_duration,closed_road_or_infinte_delay_count_2w,closed_road_or_infinte_delay_count_4w,closed_road_or_infinte_delay_count_6w,moderate_count_2w,moderate_count_4w,moderate_count_6w,major_count_2w,major_count_4w,major_count_6w,Port_wait_time_mean_2w,Port_wait_time_median_2w,Port_wait_time_max_2w,Port_wait_time_mean_4w,Port_wait_time_median_4w,Port_wait_time_max_4w,Port_wait_time_mean_6w,Port_wait_time_median_6w,Port_wait_time_max_6w
0,5259,us pl little chute,100935737,wabasha,mn,us,4565760881,2021-04-05,2021-05-06,31,2021-04-19,2021-05-03,2021-05-17,"wabasha,mn",14,0.489857,0.907143,3.229429,1.455000,1.455000,1.455000,59.971429,63.785714,69.721429,29.414286,29.414286,29.414286,12.935714,13.585714,13.585714,0.0,0.0,0.0,1.0,1.0,1.0,1.5,2.5,4.5,4,5,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,12.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.5,168.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5259,us pl little chute,100935737,wabasha,mn,us,4565760889,2021-03-19,2021-04-18,30,2021-04-02,2021-04-16,2021-04-30,"wabasha,mn",11,1.397000,1.397000,1.397000,1.232857,1.232857,1.455000,53.564286,59.971429,63.785714,30.414286,30.414286,29.414286,13.035714,13.035714,13.035714,0.0,0.0,0.0,3.5,3.5,3.5,2.0,2.0,2.5,3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [0]:
#model data with max,median and mean port congestion data is generated for 2,4 and 6 weeks for building the leadtime prediction models
for country,region,PO_week,PO_2week,PO_4week,PO_6week,indexs in zip(model_data_final['updated_VS_country'],model_data_final['updated_VS_region'],model_data_final['PO_Create_Date'].dt.week,model_data_final['PO_create_date_2W'].dt.week,model_data_final['PO_create_date_4W'].dt.week,model_data_final['PO_create_date_6W'].dt.week,model_data_final.index):
  if country=='us': #(vendors within US)
    if PO_week<PO_2week:
      model_data_final['Port_wait_time_mean_2w'][indexs]=port_data[(port_data['PortState']==region) & (port_data['Week']>=PO_week) & (port_data['Week']<=PO_2week)]['TotalWait'].mean()
      model_data_final['Port_wait_time_median_2w'][indexs]=port_data[(port_data['PortState']==region) & (port_data['Week']>=PO_week) & (port_data['Week']<=PO_2week)]['TotalWait'].median()
      model_data_final['Port_wait_time_max_2w'][indexs]=port_data[(port_data['PortState']==region) & (port_data['Week']>=PO_week) & (port_data['Week']<=PO_2week)]['TotalWait'].max()
    else:
      model_data_final['Port_wait_time_mean_2w'][indexs]=port_data[(port_data['PortState']==region) & ((port_data['Week']>=PO_week) | (port_data['Week']<=PO_2week))]['TotalWait'].mean()
      model_data_final['Port_wait_time_median_2w'][indexs]=port_data[(port_data['PortState']==region) & ((port_data['Week']>=PO_week) | (port_data['Week']<=PO_2week))]['TotalWait'].median()
      model_data_final['Port_wait_time_max_2w'][indexs]=port_data[(port_data['PortState']==region) & ((port_data['Week']>=PO_week) | (port_data['Week']<=PO_2week))]['TotalWait'].max()
    if PO_week<PO_4week:
      model_data_final['Port_wait_time_mean_4w'][indexs]=port_data[(port_data['PortState']==region) & (port_data['Week']>=PO_week) & (port_data['Week']<=PO_4week)]['TotalWait'].mean()
      model_data_final['Port_wait_time_median_4w'][indexs]=port_data[(port_data['PortState']==region) & (port_data['Week']>=PO_week) & (port_data['Week']<=PO_4week)]['TotalWait'].median()
      model_data_final['Port_wait_time_max_4w'][indexs]=port_data[(port_data['PortState']==region) & (port_data['Week']>=PO_week) & (port_data['Week']<=PO_4week)]['TotalWait'].max()
    else:
      model_data_final['Port_wait_time_mean_4w'][indexs]=port_data[(port_data['PortState']==region) & ((port_data['Week']>=PO_week) | (port_data['Week']<=PO_4week))]['TotalWait'].mean()
      model_data_final['Port_wait_time_median_4w'][indexs]=port_data[(port_data['PortState']==region) & ((port_data['Week']>=PO_week) | (port_data['Week']<=PO_4week))]['TotalWait'].median()
      model_data_final['Port_wait_time_max_4w'][indexs]=port_data[(port_data['PortState']==region) & ((port_data['Week']>=PO_week) | (port_data['Week']<=PO_4week))]['TotalWait'].max()
    if PO_week<PO_6week:
      model_data_final['Port_wait_time_mean_6w'][indexs]=port_data[(port_data['PortState']==region) & (port_data['Week']>=PO_week) & (port_data['Week']<=PO_6week)]['TotalWait'].mean()
      model_data_final['Port_wait_time_median_6w'][indexs]=port_data[(port_data['PortState']==region) & (port_data['Week']>=PO_week) & (port_data['Week']<=PO_6week)]['TotalWait'].median()
      model_data_final['Port_wait_time_max_6w'][indexs]=port_data[(port_data['PortState']==region) & (port_data['Week']>=PO_week) & (port_data['Week']<=PO_6week)]['TotalWait'].max()
    else:
      model_data_final['Port_wait_time_mean_6w'][indexs]=port_data[(port_data['PortState']==region) & ((port_data['Week']>=PO_week) | (port_data['Week']<=PO_6week))]['TotalWait'].mean()
      model_data_final['Port_wait_time_median_6w'][indexs]=port_data[(port_data['PortState']==region) & ((port_data['Week']>=PO_week) | (port_data['Week']<=PO_6week))]['TotalWait'].median()
      model_data_final['Port_wait_time_max_6w'][indexs]=port_data[(port_data['PortState']==region) & ((port_data['Week']>=PO_week) | (port_data['Week']<=PO_6week))]['TotalWait'].max()
    
    
  elif ((country=='au') | (country=='vt') | (country=='tw')): #(vendors in APAC)
    if PO_week<PO_2week:
      model_data_final['Port_wait_time_mean_2w'][indexs]=port_data[((port_data['Country']==country) |(port_data['Coast']=='West Coast'))& (port_data['Week']>=PO_week) & (port_data['Week']<=PO_2week)]['TotalWait'].mean()
      model_data_final['Port_wait_time_median_2w'][indexs]=port_data[((port_data['Country']==country) |(port_data['Coast']=='West Coast'))& (port_data['Week']>=PO_week) & (port_data['Week']<=PO_2week)]['TotalWait'].median()
      model_data_final['Port_wait_time_max_2w'][indexs]=port_data[((port_data['Country']==country) |(port_data['Coast']=='West Coast'))& (port_data['Week']>=PO_week) & (port_data['Week']<=PO_2week)]['TotalWait'].max()
    else:
      model_data_final['Port_wait_time_mean_2w'][indexs]=port_data[((port_data['Country']==country) |(port_data['Coast']=='West Coast'))& ((port_data['Week']>=PO_week) | (port_data['Week']<=PO_2week))]['TotalWait'].mean()
      model_data_final['Port_wait_time_median_2w'][indexs]=port_data[((port_data['Country']==country) |(port_data['Coast']=='West Coast'))& ((port_data['Week']>=PO_week)|(port_data['Week']<=PO_2week))]['TotalWait'].median()
      model_data_final['Port_wait_time_max_2w'][indexs]=port_data[((port_data['Country']==country) |(port_data['Coast']=='West Coast'))& ((port_data['Week']>=PO_week) | (port_data['Week']<=PO_2week))]['TotalWait'].max()
    if PO_week<PO_4week:
      model_data_final['Port_wait_time_mean_4w'][indexs]=port_data[((port_data['Country']==country) |(port_data['Coast']=='West Coast'))& (port_data['Week']>=PO_week) & (port_data['Week']<=PO_4week)]['TotalWait'].mean()
      model_data_final['Port_wait_time_median_4w'][indexs]=port_data[((port_data['Country']==country) |(port_data['Coast']=='West Coast'))& (port_data['Week']>=PO_week) & (port_data['Week']<=PO_4week)]['TotalWait'].median()
      model_data_final['Port_wait_time_max_4w'][indexs]=port_data[((port_data['Country']==country) |(port_data['Coast']=='West Coast'))& (port_data['Week']>=PO_week) & (port_data['Week']<=PO_4week)]['TotalWait'].max()
    else:
      model_data_final['Port_wait_time_mean_4w'][indexs]=port_data[((port_data['Country']==country) |(port_data['Coast']=='West Coast'))& ((port_data['Week']>=PO_week) | (port_data['Week']<=PO_4week))]['TotalWait'].mean()
      model_data_final['Port_wait_time_median_4w'][indexs]=port_data[((port_data['Country']==country) |(port_data['Coast']=='West Coast'))& ((port_data['Week']>=PO_week)|(port_data['Week']<=PO_4week))]['TotalWait'].median()
      model_data_final['Port_wait_time_max_4w'][indexs]=port_data[((port_data['Country']==country) |(port_data['Coast']=='West Coast'))& ((port_data['Week']>=PO_week) | (port_data['Week']<=PO_4week))]['TotalWait'].max()
    if PO_week<PO_6week:
      model_data_final['Port_wait_time_mean_6w'][indexs]=port_data[((port_data['Country']==country) |(port_data['Coast']=='West Coast'))& (port_data['Week']>=PO_week) & (port_data['Week']<=PO_6week)]['TotalWait'].mean()
      model_data_final['Port_wait_time_median_6w'][indexs]=port_data[((port_data['Country']==country) |(port_data['Coast']=='West Coast'))& (port_data['Week']>=PO_week) & (port_data['Week']<=PO_6week)]['TotalWait'].median()
      model_data_final['Port_wait_time_max_6w'][indexs]=port_data[((port_data['Country']==country) |(port_data['Coast']=='West Coast'))& (port_data['Week']>=PO_week) & (port_data['Week']<=PO_6week)]['TotalWait'].max()
    else:
      model_data_final['Port_wait_time_mean_6w'][indexs]=port_data[((port_data['Country']==country) |(port_data['Coast']=='West Coast'))& ((port_data['Week']>=PO_week) | (port_data['Week']<=PO_6week))]['TotalWait'].mean()
      model_data_final['Port_wait_time_median_6w'][indexs]=port_data[((port_data['Country']==country) |(port_data['Coast']=='West Coast'))& ((port_data['Week']>=PO_week)|(port_data['Week']<=PO_6week))]['TotalWait'].median()
      model_data_final['Port_wait_time_max_6w'][indexs]=port_data[((port_data['Country']==country) |(port_data['Coast']=='West Coast'))& ((port_data['Week']>=PO_week) | (port_data['Week']<=PO_6week))]['TotalWait'].max()

    
  elif ((country=='es') | (country=='br') | (country=='de')): #(vendors in South America and EU)
    if PO_week<PO_2week:
      model_data_final['Port_wait_time_mean_2w'][indexs]=port_data[((port_data['Country']==country) |(port_data['Coast']=='East Coast'))& (port_data['Week']>=PO_week) & (port_data['Week']<=PO_2week)]['TotalWait'].mean()
      model_data_final['Port_wait_time_median_2w'][indexs]=port_data[((port_data['Country']==country) |(port_data['Coast']=='East Coast'))& (port_data['Week']>=PO_week) & (port_data['Week']<=PO_2week)]['TotalWait'].median()
      model_data_final['Port_wait_time_max_2w'][indexs]=port_data[((port_data['Country']==country) |(port_data['Coast']=='East Coast'))& (port_data['Week']>=PO_week) & (port_data['Week']<=PO_2week)]['TotalWait'].max()
    else:
      model_data_final['Port_wait_time_mean_2w'][indexs]=port_data[((port_data['Country']==country) |(port_data['Coast']=='East Coast'))& ((port_data['Week']>=PO_week) | (port_data['Week']<=PO_2week))]['TotalWait'].mean()
      model_data_final['Port_wait_time_median_2w'][indexs]=port_data[((port_data['Country']==country) |(port_data['Coast']=='East Coast'))& ((port_data['Week']>=PO_week)|(port_data['Week']<=PO_2week))]['TotalWait'].median()
      model_data_final['Port_wait_time_max_2w'][indexs]=port_data[((port_data['Country']==country) |(port_data['Coast']=='East Coast'))& ((port_data['Week']>=PO_week) | (port_data['Week']<=PO_2week))]['TotalWait'].max()
    if PO_week<PO_4week:
      model_data_final['Port_wait_time_mean_4w'][indexs]=port_data[((port_data['Country']==country) |(port_data['Coast']=='East Coast'))& (port_data['Week']>=PO_week) & (port_data['Week']<=PO_4week)]['TotalWait'].mean()
      model_data_final['Port_wait_time_median_4w'][indexs]=port_data[((port_data['Country']==country) |(port_data['Coast']=='East Coast'))& (port_data['Week']>=PO_week) & (port_data['Week']<=PO_4week)]['TotalWait'].median()
      model_data_final['Port_wait_time_max_4w'][indexs]=port_data[((port_data['Country']==country) |(port_data['Coast']=='East Coast'))& (port_data['Week']>=PO_week) & (port_data['Week']<=PO_4week)]['TotalWait'].max()
    else:
      model_data_final['Port_wait_time_mean_4w'][indexs]=port_data[((port_data['Country']==country) |(port_data['Coast']=='East Coast'))& ((port_data['Week']>=PO_week) | (port_data['Week']<=PO_4week))]['TotalWait'].mean()
      model_data_final['Port_wait_time_median_4w'][indexs]=port_data[((port_data['Country']==country) |(port_data['Coast']=='East Coast'))& ((port_data['Week']>=PO_week)|(port_data['Week']<=PO_4week))]['TotalWait'].median()
      model_data_final['Port_wait_time_max_4w'][indexs]=port_data[((port_data['Country']==country) |(port_data['Coast']=='East Coast'))& ((port_data['Week']>=PO_week) | (port_data['Week']<=PO_4week))]['TotalWait'].max()
    if PO_week<PO_6week:
      model_data_final['Port_wait_time_mean_6w'][indexs]=port_data[((port_data['Country']==country) |(port_data['Coast']=='East Coast'))& (port_data['Week']>=PO_week) & (port_data['Week']<=PO_6week)]['TotalWait'].mean()
      model_data_final['Port_wait_time_median_6w'][indexs]=port_data[((port_data['Country']==country) |(port_data['Coast']=='East Coast'))& (port_data['Week']>=PO_week) & (port_data['Week']<=PO_6week)]['TotalWait'].median()
      model_data_final['Port_wait_time_max_6w'][indexs]=port_data[((port_data['Country']==country) |(port_data['Coast']=='East Coast'))& (port_data['Week']>=PO_week) & (port_data['Week']<=PO_6week)]['TotalWait'].max()
    else:
      model_data_final['Port_wait_time_mean_6w'][indexs]=port_data[((port_data['Country']==country) |(port_data['Coast']=='East Coast'))& ((port_data['Week']>=PO_week) | (port_data['Week']<=PO_6week))]['TotalWait'].mean()
      model_data_final['Port_wait_time_median_6w'][indexs]=port_data[((port_data['Country']==country) |(port_data['Coast']=='East Coast'))& ((port_data['Week']>=PO_week)|(port_data['Week']<=PO_6week))]['TotalWait'].median()
      model_data_final['Port_wait_time_max_6w'][indexs]=port_data[((port_data['Country']==country) |(port_data['Coast']=='East Coast'))& ((port_data['Week']>=PO_week) | (port_data['Week']<=PO_6week))]['TotalWait'].max()
    
print(model_data_final.shape)
model_data_final.head()

(69351, 147)
Out[36]:

,Plant_ID,Plant_Name,updated_VS_ID,updated_VS_location,updated_VS_region,updated_VS_country,Purchase_Order,PO_Create_Date,Delivery_date,lead_time,PO_create_date_2W,PO_create_date_4W,PO_create_date_6W,name,Weeks,rainfall_mm_2W,rainfall_mm_4W,rainfall_mm_6W,snowdepth_2W,snowdepth_4W,snowdepth_6W,tempmax_2W,tempmax_4W,tempmax_6W,tempmin_2W,tempmin_4W,tempmin_6W,windspeed_2W,windspeed_4W,windspeed_6W,fog_count_2W,fog_count_4W,fog_count_6W,snow_count_2W,snow_count_4W,snow_count_6W,rain_count_2W,rain_count_4W,rain_count_6W,PO_month,PO_month_6W,extreme_cold/wind_chill_count,extreme_cold/wind_chill_duration,thunderstorm_wind_count,thunderstorm_wind_duration,winter_weather_count,winter_weather_duration,tropical_depression_count,tropical_depression_duration,hurricane_count,hurricane_duration,funnel_cloud_count,funnel_cloud_duration,freezing_fog_count,freezing_fog_duration,cold/wind_chill_count,cold/wind_chill_duration,debris_flow_count,debris_flow_duration,strong_wind_count,strong_wind_duration,high_wind_count,high_wind_duration,lakeshore_flood_count,lakeshore_flood_duration,dense_smoke_count,dense_smoke_duration,frost/freeze_count,frost/freeze_duration,dust_storm_count,dust_storm_duration,tornado_count,tornado_duration,lake-effect_snow_count,lake-effect_snow_duration,storm_surge/tide_count,storm_surge/tide_duration,drought_count,drought_duration,blizzard_count,blizzard_duration,tropical_storm_count,tropical_storm_duration,coastal_flood_count,coastal_flood_duration,wildfire_count,wildfire_duration,high_surf_count,high_surf_duration,dense_fog_count,dense_fog_duration,lightning_count,lightning_duration,excessive_heat_count,excessive_heat_duration,seiche_count,seiche_duration,avalanche_count,avalanche_duration,heavy_rain_count,heavy_rain_duration,ice_storm_count,ice_storm_duration,heavy_snow_count,heavy_snow_duration,winter_storm_count,winter_storm_duration,heat_count,heat_duration,sleet_count,sleet_duration,flash_flood_count,flash_flood_duration,hail_count,hail_duration,rip_current_count,rip_current_duration,flood_count,flood_duration,marine_dense_fog_count,marine_dense_fog_duration,marine_high_wind_count,marine_high_wind_duration,marine_thunderstorm_wind_count,marine_thunderstorm_wind_duration,marine_tropical_storm_count,marine_tropical_storm_duration,waterspout_count,waterspout_duration,closed_road_or_infinte_delay_count_2w,closed_road_or_infinte_delay_count_4w,closed_road_or_infinte_delay_count_6w,moderate_count_2w,moderate_count_4w,moderate_count_6w,major_count_2w,major_count_4w,major_count_6w,Port_wait_time_mean_2w,Port_wait_time_median_2w,Port_wait_time_max_2w,Port_wait_time_mean_4w,Port_wait_time_median_4w,Port_wait_time_max_4w,Port_wait_time_mean_6w,Port_wait_time_median_6w,Port_wait_time_max_6w
0,5259,us pl little chute,100935737,wabasha,mn,us,4565760881,2021-04-05,2021-05-06,31,2021-04-19,2021-05-03,2021-05-17,"wabasha,mn",14,0.489857,0.907143,3.229429,1.455000,1.455000,1.455000,59.971429,63.785714,69.721429,29.414286,29.414286,29.414286,12.935714,13.585714,13.585714,0.0,0.0,0.0,1.0,1.0,1.0,1.5,2.5,4.5,4,5,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,12.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.5,168.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5259,us pl little chute,100935737,wabasha,mn,us,4565760889,2021-03-19,2021-04-18,30,2021-04-02,2021-04-16,2021-04-30,"wabasha,mn",11,1.397000,1.397000,1.397000,1.232857,1.232857,1.455000,53.564286,59.971429,63.785714,30.414286,30.414286,29.414286,13.035714,13.035714,13.035714,0.0,0.0,0.0,3.5,3.5,3.5,2.0,2.0,2.5,3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [0]:
#saving the model with Port congestion data
col_names = ['_'.join(col.split(' ')) for col in model_data_final.columns.tolist()]
model_data_final.columns = col_names
model_data_w_weather_traffic_port = spark.createDataFrame(model_data_final)
model_data_w_weather_traffic_port.write.saveAsTable(Model_with_weather_traffic_port_dataset,mode = 'overwrite')

In [0]:
#Reading internal NUSA analytical data from phase 1
final_data = spark.read.table('model_data_vf_19_01')  #enter table name
final_data = final_data.toPandas()
final_data = final_data[final_data['PO_Create_Date']!='1888-01-01']
final_data = final_data[final_data['Delivered_Quantity']!=0]
print(final_data.shape)
final_data.head()

(69351, 103)
Out[38]:

,Plant_ID,Plant_Name,Vendor,VM_Name,Purchase_Order,PO_Item,Material_Group,Material_No.,Material_Description,First_GR_Date,Last_GR_Date,First_Statistical_delivery_Date,Delivery_date,Purchase_Order_Scheduled_Qty,Delivered_Quantity,Purchase_order_PO_item,Missed_PO,PO_Create_Date,updated_VS_country,updated_VS_ID,updated_VS_location,updated_VS_region,distance,vendor_percentage,vendor_material_cnt_per_plant,sku_type_single,dom_or_int_int,Material_group_type_R,nestle_managed_freight_yes,mode_of_transport_road,single_or_multisource_single,PO_Create_Month_1,PO_Create_Month_2,PO_Create_Month_3,PO_Create_Month_4,PO_Create_Month_5,PO_Create_Month_6,PO_Create_Month_7,PO_Create_Month_8,PO_Create_Month_9,PO_Create_Month_10,PO_Create_Month_11,PO_Create_Month_12,bus_day_count,lead_time,holiday_weekend_count,OTIF,l3m_date,l12m_date,no.of.complaint_3_months,no.of.complaint_12_months,no.of.complaint_3_months_major,no.of.complaint_3_months_minor,no.of.complaint_12_months_major,no.of.complaint_12_months_minor,no.of.complaint_3_months_open,no.of.complaint_3_months_open_major,no.of.complaint_3_months_open_minor,no.of.Complaint_type_Documentation_l12m,Complaint_type_Documentation_l12m_distinct,no.of.Complaint_type_Packaging_l12m,Complaint_type_Packaging_l12m_distinct,no.of.Complaint_type_Transportation_l12m,Complaint_type_Transportation_l12m_distinct,no.of.Complaint_type_Ingredient_l12m,Complaint_type_Ingredient_l12m_distinct,no.of.Complaint_type_Consumer_l12m,Complaint_type_Consumer_l12m_distinct,no.of.Complaint_type_Foreign_Material_l12m,Complaint_type_Foreign_Material_l12m_distinct,no.of.Complaint_type_Indigenous_Material_l12m,Complaint_type_Indigenous_Material_l12m_distinct,Risk_Level_High,Risk_Level_Low,Risk_Level_Medium,Rating_A,Rating_B,Rating_C,Rating_D,Rating_D_-_FND,Rating_F,Rating_Inactive,Rating_Out_of_Scope,days,Delivered_Quantity_UoM,Type_of_Measure,MovingAverage,POqty_to_AvgDelQty,range_bin_7_to_70,range_bin_70_to_217,range_bin_217_to_326,range_bin_326_to_544,range_bin_544_to_904,range_bin_904_to_1288,range_bin_1288_to_2153,range_bin_2153_to_2874,range_bin_2874_to_15463,Delivery_freq_Daily,Delivery_freq_Half_Yearly,Delivery_freq_Monthly,Delivery_freq_Quarterly,Delivery_freq_Weekly,Delivery_freq_Yearly
0,5259,us pl little chute,100037407,nestrade s.a.,4565760881,10,cereals & grains,22019000,flour wheat tae 12.5%prtn 0.60%ash ench,2021-05-02,2021-05-02,2021-05-06,2021-05-06,200000.0,214996.0,4565760881_10,No,2021-04-05,us,100935737,wabasha,mn,377.8423,10.84,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,24,31,7,1.0,2021-01-05,2020-04-05,13,43,0,13,7,36,0,0,0,6,1,0,0,0,0,1,1,0,0,35,1,1,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,13.391304,LB,mass,212148.00,0.942738,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
1,5259,us pl little chute,100037407,nestrade s.a.,4565760889,10,cereals & grains,22019000,flour wheat tae 12.5%prtn 0.60%ash ench,2021-04-15,2021-04-15,2021-04-18,2021-04-18,200000.0,189974.0,4565760889_10,No,2021-03-19,us,100935737,wabasha,mn,377.8423,10.84,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,21,30,9,1.0,2020-12-19,2020-03-19,13,40,0,13,8,32,0,0,0,7,1,0,0,0,0,0,0,0,0,32,1,1,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,13.391304,LB,mass,155702.75,1.284499,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
2,5259,us pl little chute,100037407,nestrade s.a.,4565762950,10,cereals & grains,22019000,flour wheat tae 12.5%prtn 0.60%ash ench,2021-04-17,2021-04-17,2021-04-27,2021-04-27,200000.0,213661.0,4565762950_10,No,2021-03-26,us,100935737,wabasha,mn,377.8423,10.84,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,23,32,9,1.0,2020-12-26,2020-03-26,13,43,0,13,8,35,0,0,0,7,1,0,0,0,0,1,1,0,0,34,1,1,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,13.391304,LB,mass,208670.50,0.958449,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
3,5259,us pl little chute,100037407,nestrade s.a.,4565768904,10,cereals & grains,22019000,flour wheat tae 12.5%prtn 0.60%ash ench,2021-04-21,2021-04-21,2021-04-30,2021-04-30,200000.0,208749.0,4565768904_10,No,2021-03-26,us,100935737,wabasha,mn,377.8423,10.84,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0

In [0]:
#Filtering out columns not required for modelling
cols_to_keep = ['Plant_ID','Plant_Name','Vendor','updated_VS_ID','updated_VS_location','updated_VS_region','updated_VS_country','Purchase_Order',
                'PO_Item','Material_No.','Material_Description','PO_Create_Date','Delivery_date','vendor_percentage', 
                'vendor_material_cnt_per_plant', 'dom_or_int_int', 'sku_type_single', 'distance', 'POqty_to_AvgDelQty', 
                'nestle_managed_freight_yes', 'mode_of_transport_road', 'Material_group_type_R', 'single_or_multisource_single', 
                'no.of.complaint_3_months', 'no.of.complaint_12_months', 'no.of.complaint_3_months_major', 
                'no.of.complaint_3_months_minor', 'no.of.complaint_12_months_major', 'no.of.complaint_12_months_minor', 
                'no.of.complaint_3_months_open', 'no.of.complaint_3_months_open_major', 'no.of.complaint_3_months_open_minor', 
                'no.of.Complaint_type_Documentation_l12m', 'no.of.Complaint_type_Packaging_l12m', 'no.of.Complaint_type_Transportation_l12m', 
                'no.of.Complaint_type_Ingredient_l12m', 'no.of.Complaint_type_Consumer_l12m', 'no.of.Complaint_type_Foreign_Material_l12m', 
                'no.of.Complaint_type_Indigenous_Material_l12m', 'Risk_Level_High', 'Risk_Level_Low', 'Risk_Level_Medium', 'Rating_A', 
                'Rating_B', 'Rating_C', 'Rating_D', 'Rating_D_-_FND', 'Rating_F', 'Rating_Inactive', 'Rating_Out_of_Scope', 
                'Delivery_freq_Daily', 'Delivery_freq_Half_Yearly', 'Delivery_freq_Monthly', 'Delivery_freq_Quarterly', 'Delivery_freq_Weekly', 
                'Delivery_freq_Yearly', 'Complaint_type_Documentation_l12m_distinct', 'Complaint_type_Packaging_l12m_distinct', 
                'Complaint_type_Transportation_l12m_distinct', 'Complaint_type_Ingredient_l12m_distinct', 
                'Complaint_type_Consumer_l12m_distinct', 'Complaint_type_Foreign_Material_l12m_distinct', 
                'Complaint_type_Indigenous_Material_l12m_distinct','lead_time', 'bus_day_count']

final_data_v1 = final_data[cols_to_keep]
final_data_v1.fillna(0, inplace=True)

In [0]:
#Updating location attributes
final_data_v1.loc[(final_data_v1['updated_VS_region']=='de') & (final_data_v1['updated_VS_country']!='us'), 'updated_VS_country'] = 'de'
final_data_v1.loc[final_data_v1['updated_VS_region']=='br', 'updated_VS_country'] = 'br'
final_data_v1.loc[final_data_v1['updated_VS_region']=='vn', 'updated_VS_country'] = 'vt'
final_data_v1.loc[final_data_v1['updated_VS_country']=='vt', 'updated_VS_region'] = 'vt'
final_data_v1.loc[final_data_v1['updated_VS_country']=='de', 'updated_VS_region'] = 'null'
final_data_v1.loc[(final_data_v1['updated_VS_location']=='madison') & 
                  (final_data_v1['updated_VS_region']=='id'), 'updated_VS_country'] = 'us'
final_data_v1.loc[(final_data_v1['updated_VS_location']=='payette') & 
                  (final_data_v1['updated_VS_region']=='id'), 'updated_VS_country'] = 'us'

In [0]:
#Reading external analytical data 
weekly_ext_vars = spark.read.table('Model_weather_traffic_port_congestion_data_v1')
weekly_ext_vars = weekly_ext_vars.toPandas()

In [0]:
#Creating list of all extreme weather variables
ext_weather_cols = ['high_surf_count', 'high_surf_duration',	'dense_smoke_count',	'dense_smoke_duration', 'frost/freeze_count', 'frost/freeze_duration', 'dense_fog_count', 'dense_fog_duration', 'tropical_depression_count',	'tropical_depression_duration', 'hurricane_count', 'hurricane_duration', 'ice_storm_count', 'ice_storm_duration', 'heavy_snow_count', 'heavy_snow_duration', 'funnel_cloud_count', 'funnel_cloud_duration', 'winter_storm_count', 'winter_storm_duration', 'heat_count', 'heat_duration', 'blizzard_count', 'blizzard_duration', 'tropical_storm_count', 'tropical_storm_duration', 'freezing_fog_count', 'freezing_fog_duration', 'cold/wind_chill_count', 'cold/wind_chill_duration', 'debris_flow_count', 'debris_flow_duration', 'lightning_count', 'lightning_duration', 'thunderstorm_wind_count', 'thunderstorm_wind_duration', 'astronomical_low_tide_count', 'astronomical_low_tide_duration', 'winter_weather_count', 'winter_weather_duration', 'hail_count', 'hail_duration', 'rip_current_count', 'rip_current_duration', 'extreme_cold/wind_chill_count', 'extreme_cold/wind_chill_duration', 'excessive_heat_count', 'excessive_heat_duration', 'marine_thunderstorm_wind_count', 'marine_thunderstorm_wind_duration', 'marine_high_wind_count', 'marine_high_wind_duration', 'coastal_flood_count', 'coastal_flood_duration', 'wildfire_count', 'wildfire_duration', 'high_wind_count', 'high_wind_duration', 'lakeshore_flood_count', 'lakeshore_flood_duration', 'flood_count', 'flood_duration', 'marine_dense_fog_count', 'marine_dense_fog_duration', 'heavy_rain_count', 'heavy_rain_duration', 'marine_tropical_storm_count', 'marine_tropical_storm_duration', 'tornado_count', 'tornado_duration', 'lake-effect_snow_count', 'lake-effect_snow_duration', 'waterspout_count', 'waterspout_duration', 'strong_wind_count', 'strong_wind_duration', 'storm_surge/tide_count', 'storm_surge/tide_duration', 'drought_count', 'drought_duration', 'sleet_count', 'sleet_duration', 'flash_flood_count', 'flash_flood_duration']

In [0]:
#Creating list of significant extreme weather event variables
imp_ext_weather_count_cols = ['winter_storm_count','flash_flood_count','heat_count','hail_count','thunderstorm_wind_count','lightning_count',
                              'high_wind_count','wildfire_count','heavy_snow_count','strong_wind_count','winter_weather_count',
                              'extreme_cold/wind_chill_count','flood_count','heavy_rain_count','blizzard_count','cold/wind_chill_count',
                              'drought_count','tornado_count','lakeshore_flood_count','frost/freeze_count','tropical_storm_count',
                              'coastal_flood_count','marine_high_wind_count','marine_thunderstorm_wind_count','storm_surge/tide_count',
                              'tropical_depression_count','dense_fog_count','hurricane_count','marine_tropical_storm_count','ice_storm_count',
                              'marine_dense_fog_count','excessive_heat_count','waterspout_count','dense_smoke_count','sleet_count']

imp_ext_weather_duration_cols = ['marine_high_wind_duration','marine_thunderstorm_wind_duration','storm_surge/tide_duration',
                                 'waterspout_duration','tropical_depression_duration','marine_tropical_storm_duration']

#Creating list of extreme weather event variables to keep from external analytical data
ext_weather_count_cols_weekly = list(set(col for col in ext_weather_cols if 'count' in col).intersection(imp_ext_weather_count_cols))
ext_weather_duration_cols_weekly = list(set(col for col in ext_weather_cols if 'duration' in col).intersection(imp_ext_weather_duration_cols))

In [0]:
#Filtering required columns from external analytical data
weekly_ext_vars_v1 = weekly_ext_vars[['Plant_ID', 'updated_VS_ID', 'Purchase_Order', 'PO_Create_Date', 'Delivery_date']+
                                     [col for col in weekly_ext_vars.columns.tolist() if '_2W' in col]+
                                     [col for col in weekly_ext_vars.columns.tolist() if '_4W' in col]+
                                     [col for col in weekly_ext_vars.columns.tolist() if '_6W' in col]+
                                     [col for col in weekly_ext_vars.columns.tolist() if 'Port_wait_time_' in col]+
                                     ['closed_road_or_infinte_delay_count_2w','closed_road_or_infinte_delay_count_4w',
                                      'closed_road_or_infinte_delay_count_6w','moderate_count_2w','moderate_count_4w','moderate_count_6w',
                                      'major_count_2w','major_count_4w','major_count_6w']+ext_weather_count_cols_weekly+
                                     ext_weather_duration_cols_weekly].drop_duplicates()

#If there have been no counts of events which are significant for predicting lead time setting the count and duration for them as 0
imp_ext_weather_cols = imp_ext_weather_count_cols+imp_ext_weather_duration_cols
for col in imp_ext_weather_cols:
  if col not in weekly_ext_vars_v1.columns.tolist():
    weekly_ext_vars_v1[col]=0

weekly_ext_vars_v1.fillna(0, inplace=True)
weekly_ext_vars_v1.drop(columns=[col for col in weekly_ext_vars_v1.columns.tolist() if col != 'PO_Create_Date' and 'PO' in col], inplace=True)
final_data_v2 = final_data_v1.drop(columns=[col for col in final_data_v1.columns.tolist() if '_exw' in col])
final_data_v3 = final_data_v2.drop(columns=[col for col in final_data_v2.columns.tolist() if '_duration' in col])

#Creating final analytical data with internal and external variables
final_analytical_data = final_data_v3.merge(weekly_ext_vars_v1, on=['Plant_ID', 'updated_VS_ID', 'Purchase_Order', 'PO_Create_Date', 
                                                                    'Delivery_date'], how='left')

In [0]:
final_analytical_data.head()

Out[64]:

,Plant_ID,Plant_Name,Vendor,updated_VS_ID,updated_VS_location,updated_VS_region,updated_VS_country,Purchase_Order,PO_Item,Material_No.,Material_Description,PO_Create_Date,Delivery_date,vendor_percentage,vendor_material_cnt_per_plant,dom_or_int_int,sku_type_single,distance,POqty_to_AvgDelQty,nestle_managed_freight_yes,mode_of_transport_road,Material_group_type_R,single_or_multisource_single,no.of.complaint_3_months,no.of.complaint_12_months,no.of.complaint_3_months_major,no.of.complaint_3_months_minor,no.of.complaint_12_months_major,no.of.complaint_12_months_minor,no.of.complaint_3_months_open,no.of.complaint_3_months_open_major,no.of.complaint_3_months_open_minor,no.of.Complaint_type_Documentation_l12m,no.of.Complaint_type_Packaging_l12m,no.of.Complaint_type_Transportation_l12m,no.of.Complaint_type_Ingredient_l12m,no.of.Complaint_type_Consumer_l12m,no.of.Complaint_type_Foreign_Material_l12m,no.of.Complaint_type_Indigenous_Material_l12m,Risk_Level_High,Risk_Level_Low,Risk_Level_Medium,Rating_A,Rating_B,Rating_C,Rating_D,Rating_D_-_FND,Rating_F,Rating_Inactive,Rating_Out_of_Scope,Delivery_freq_Daily,Delivery_freq_Half_Yearly,Delivery_freq_Monthly,Delivery_freq_Quarterly,Delivery_freq_Weekly,Delivery_freq_Yearly,Complaint_type_Documentation_l12m_distinct,Complaint_type_Packaging_l12m_distinct,Complaint_type_Transportation_l12m_distinct,Complaint_type_Ingredient_l12m_distinct,Complaint_type_Consumer_l12m_distinct,Complaint_type_Foreign_Material_l12m_distinct,Complaint_type_Indigenous_Material_l12m_distinct,lead_time,bus_day_count,rainfall_mm_2W,snowdepth_2W,tempmax_2W,tempmin_2W,windspeed_2W,fog_count_2W,snow_count_2W,rain_count_2W,rainfall_mm_4W,snowdepth_4W,tempmax_4W,tempmin_4W,windspeed_4W,fog_count_4W,snow_count_4W,rain_count_4W,rainfall_mm_6W,snowdepth_6W,tempmax_6W,tempmin_6W,windspeed_6W,fog_count_6W,snow_count_6W,rain_count_6W,Port_wait_time_mean_2w,Port_wait_time_median_2w,Port_wait_time_max_2w,Port_wait_time_mean_4w,Port_wait_time_median_4w,Port_wait_time_max_4w,Port_wait_time_mean_6w,Port_wait_time_median_6w,Port_wait_time_max_6w,closed_road_or_infinte_delay_count_2w,closed_road_or_infinte_delay_count_4w,closed_road_or_infinte_delay_count_6w,moderate_count_2w,moderate_count_4w,moderate_count_6w,major_count_2w,major_count_4w,major_count_6w,coastal_flood_count,high_wind_count,dense_smoke_count,tropical_storm_count,wildfire_count,winter_weather_count,blizzard_count,cold/wind_chill_count,tropical_depression_count,hurricane_count,heavy_rain_count,marine_tropical_storm_count,lakeshore_flood_count,sleet_count,drought_count,marine_dense_fog_count,waterspout_count,frost/freeze_count,thunderstorm_wind_count,dense_fog_count,strong_wind_count,marine_thunderstorm_wind_count,tornado_count,hail_count,storm_surge/tide_count,marine_high_wind_count,winter_storm_count,extreme_cold/wind_chill_count,heat_count,lightning_count,excessive_heat_count,flash_flood_count,flood_count,heavy_snow_count,ice_storm_count,waterspout_duration,storm_surge/tide_duration,marine_high_wind_duration,marine_tropical_storm_duration,tropical_depression_duration,marine_thunderstorm_wind_duration
0,5259,us pl little chute,100037407,100935737,wabasha,mn,us,4565760881,10,22019000,flour wheat tae 12.5%prtn 0.60%ash ench,2021-04-05,2021-05-06,10.84,1,0,0,377.8423,0.942738,0,0,1,0,13,43,0,13,7,36,0,0,0,6,0,0,1,0,35,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,0,0,1,0,1,0,0,1,0,1,1,31,24,0.489857,1.455000,59.971429,29.414286,12.935714,0.0,1.0,1.5,0.907143,1.455000,63.785714,29.414286,13.585714,0.0,1.0,2.5,3.229429,1.455000,69.721429,29.414286,13.585714,0.0,1.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5259,us pl little chute,100037407,100935737,wabasha,mn,us,4565760889,10,22019000,flour wheat tae 12.5%prtn 0.60%ash ench,2021-03-19,2021-04-18,10.84,1,0,0,377.8423,1.28449

In [0]:
#Saving final analytical data
analytical_data_int_ext = spark.createDataFrame(final_analytical_data)
analytical_data_int_ext.write.saveAsTable('analytical_data_int_ext_04_03_v1',mode = 'overwrite')